## 1. Importations

In [5]:
import pandas as pd

from src.modeling import train_and_track_model_with_cv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
import mlflow

import warnings
import logging

# Masquer les warnings MLflow liés à l'environnement
warnings.filterwarnings("ignore", message=".*Failed to resolve installed pip version.*")
logging.getLogger("mlflow.utils.environment").setLevel(logging.ERROR)
mlflow.set_tracking_uri("http://localhost:5001")

## 2. Chargement des données

In [6]:
DATA_PATH = '../datas/02_preprocess/datas.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print(f"Données chargées. Forme: {df.shape}")
except FileNotFoundError:
    print(f"Erreur: Le fichier {DATA_PATH} n'a pas été trouvé. Vérifier le chemin.")

# Séparation des features (X) et de la cible (y)
if 'TARGET' in df.columns:
    X = df.drop('TARGET', axis=1)
    y = df['TARGET']
    print(f"X shape: {X.shape}, y shape: {y.shape}")
else:
    print("Erreur: La colonne 'TARGET' n'a pas été trouvée. Vérifier le nom de la colonne cible.")

Données chargées. Forme: (307507, 139)
X shape: (307507, 138), y shape: (307507,)


## 3. Simple séparation

In [7]:
# Division initiale pour avoir un ensemble de test final non touché par la CV
# Nous utiliserons X_train_full et y_train_full pour la validation croisée
X_train_full, X_test_final, y_train_full, y_test_final = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Taille de l'ensemble d'entraînement : {X_train_full.shape[0]} échantillons")
print(f"Taille de l'ensemble de test : {X_test_final.shape[0]} échantillons")
print(f"Proportion de départs dans y_train : {y_train_full.mean():.2%}")
print(f"Proportion de départs dans y_test : {y_test_final.mean():.2%}")

Taille de l'ensemble d'entraînement : 246005 échantillons
Taille de l'ensemble de test : 61502 échantillons
Proportion de départs dans y_train : 8.07%
Proportion de départs dans y_test : 8.07%


## 4. Expérimentation par modèle

# Comparaison de modèles et validation croisée

Objectifs:
- Comparer plusieurs algorithmes de classification sur des données déséquilibrées.
- Utiliser une StratifiedKFold pour une évaluation robuste.
- Logger systématiquement les performances et artefacts dans MLflow.

Modèles testés:
- LogisticRegression (avec/without scaler, class_weight='balanced')
- RandomForest (class_weight='balanced')
- XGBoost / LightGBM (gestion native du déséquilibre)

Métriques suivies:
- ROC AUC, F1, Recall, Precision, PR AUC

Déséquilibre des classes:
- Class weights (selon le modèle)
- Option SMOTE (pipeline imblearn) quand pertinent

Sorties:
- Récapitulatif des performances (moyenne/écart CV)
- Artefacts: matrice de confusion, importance des features (arbres)
- Modèles enregistrés dans MLflow Model Registry

In [8]:
experiment_name = "Credit_Scoring_Comparison_CV"
mlflow.set_experiment(experiment_name)
print(f"MLflow Experiment set to: {experiment_name}")

MLflow Experiment set to: Credit_Scoring_Comparison_CV


In [9]:
all_models_avg_metrics = {}

In [10]:
# --- Modèle 1: Régression Logistique ---
lr_params = {
    'solver': 'lbfgs',
    'max_iter': 2000,
    'C': 0.1,
    'class_weight': 'balanced'
}

lr_avg_metrics = train_and_track_model_with_cv(
    "LogisticRegression", LogisticRegression, lr_params, X_train_full, y_train_full, apply_scaler=True
)

all_models_avg_metrics["LogisticRegression_Scaled"] = lr_avg_metrics


--- Démarrage de l'expérimentation pour LogisticRegression ---
  LogisticRegression - Fold 1 metrics logged. ROC AUC: 0.7432
🏃 View run LogisticRegression_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/6924ce8d4cba4bccaf378bc5e09c6c1d
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LogisticRegression - Fold 2 metrics logged. ROC AUC: 0.7411
🏃 View run LogisticRegression_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/4635d8f0b47740f2ba07f887cedf2ff7
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LogisticRegression - Fold 3 metrics logged. ROC AUC: 0.7408
🏃 View run LogisticRegression_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/1b96ae834a324848bac108b30acd29d4
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LogisticRegression - Fold 4 metrics logged. ROC AUC: 0.7468
🏃 View run LogisticRegression_Fold_4 at: http://localho

Registered model 'LogisticRegression_CreditScoring_CV_Scaler_True' already exists. Creating a new version of this model...
2025/12/05 14:07:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression_CreditScoring_CV_Scaler_True, version 3


Pas de feature_importances_ disponible.
🏃 View run LogisticRegression_CV_Run_Scaler_True at: http://localhost:5001/#/experiments/672741076646558312/runs/c38e249353b54a299ccc5ae91bda2f19
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour LogisticRegression terminée et loggée dans MLflow. ---


Created version '3' of model 'LogisticRegression_CreditScoring_CV_Scaler_True'.


In [11]:
# --- Modèle 1: Régression Logistique ---
lr_params = {
    'solver': 'lbfgs',
    'max_iter': 2000,
    'C': 0.1,
    'class_weight': 'balanced'
}
lr_avg_metrics = train_and_track_model_with_cv(
    "LogisticRegression",
    LogisticRegression,
    lr_params,
    X_train_full, y_train_full,
    apply_scaler=True,
    use_smote=True   # << actif pour LR
)
all_models_avg_metrics["LogisticRegression_Scaled_SMOTE"] = lr_avg_metrics


--- Démarrage de l'expérimentation pour LogisticRegression ---
  LogisticRegression - Fold 1 metrics logged. ROC AUC: 0.6830
🏃 View run LogisticRegression_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/0531eda92e1f42c4ba2708f2f178e072
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LogisticRegression - Fold 2 metrics logged. ROC AUC: 0.6807
🏃 View run LogisticRegression_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/ac83c0832d6e4becb851d44fd52b3645
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LogisticRegression - Fold 3 metrics logged. ROC AUC: 0.6808
🏃 View run LogisticRegression_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/5b2f077f7dc34231b3f97dec4b81b603
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LogisticRegression - Fold 4 metrics logged. ROC AUC: 0.6828
🏃 View run LogisticRegression_Fold_4 at: http://localho

Registered model 'LogisticRegression_CreditScoring_CV_Scaler_True' already exists. Creating a new version of this model...
2025/12/05 14:08:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression_CreditScoring_CV_Scaler_True, version 4


Pas de feature_importances_ disponible.
🏃 View run LogisticRegression_CV_Run_Scaler_True at: http://localhost:5001/#/experiments/672741076646558312/runs/b8c82413afa3422d80d95bb5e9c2d015
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour LogisticRegression terminée et loggée dans MLflow. ---


Created version '4' of model 'LogisticRegression_CreditScoring_CV_Scaler_True'.


In [12]:
# --- Modèle 2: Random Forest (SANS StandardScaler - généralement pas nécessaire pour les arbres) ---
rf_params = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_leaf': 5,
    'class_weight': 'balanced',
    'n_jobs': -1
}
rf_avg_metrics = train_and_track_model_with_cv(
    "RandomForestClassifier", RandomForestClassifier, rf_params, X_train_full, y_train_full, apply_scaler=False
)
all_models_avg_metrics["RandomForestClassifier_NoScaler"] = rf_avg_metrics


--- Démarrage de l'expérimentation pour RandomForestClassifier ---
  RandomForestClassifier - Fold 1 metrics logged. ROC AUC: 0.7412
🏃 View run RandomForestClassifier_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/245633658d3b44fcb516a045b82fbeca
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  RandomForestClassifier - Fold 2 metrics logged. ROC AUC: 0.7391
🏃 View run RandomForestClassifier_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/7de73c3e61fe497b88a01212fe3de161
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  RandomForestClassifier - Fold 3 metrics logged. ROC AUC: 0.7377
🏃 View run RandomForestClassifier_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/d21efed085eb40038648b113c93ea81a
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  RandomForestClassifier - Fold 4 metrics logged. ROC AUC: 0.7422
🏃 View run RandomForest

Registered model 'RandomForestClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/12/05 14:08:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier_CreditScoring_CV_Scaler_False, version 3
Created version '3' of model 'RandomForestClassifier_CreditScoring_CV_Scaler_False'.


Feature importance loggée: feature_importance.png
🏃 View run RandomForestClassifier_CV_Run_Scaler_False at: http://localhost:5001/#/experiments/672741076646558312/runs/e8340a5d0a354668b4c40193eb23b1e5
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour RandomForestClassifier terminée et loggée dans MLflow. ---


In [13]:
# --- Modèle 2: Random Forest ---
rf_params = {
    'n_estimators': 200,
    'max_depth': 10,
    'min_samples_leaf': 5,
    'n_jobs': -1
    # class_weight retiré si SMOTE activé
}
rf_avg_metrics = train_and_track_model_with_cv(
    "RandomForestClassifier",
    RandomForestClassifier,
    rf_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=True   # << actif pour RF
)
all_models_avg_metrics["RandomForestClassifier_SMOTE"] = rf_avg_metrics


--- Démarrage de l'expérimentation pour RandomForestClassifier ---
  RandomForestClassifier - Fold 1 metrics logged. ROC AUC: 0.6707
🏃 View run RandomForestClassifier_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/b51cd68543d242dbabac3ff376c65562
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  RandomForestClassifier - Fold 2 metrics logged. ROC AUC: 0.6764
🏃 View run RandomForestClassifier_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/e8e257c4b9eb46679fd30769fe150419
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  RandomForestClassifier - Fold 3 metrics logged. ROC AUC: 0.6738
🏃 View run RandomForestClassifier_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/b9432199b0134d9489ec4fea8c7a2cce
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  RandomForestClassifier - Fold 4 metrics logged. ROC AUC: 0.6817
🏃 View run RandomForest

Registered model 'RandomForestClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/12/05 14:10:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier_CreditScoring_CV_Scaler_False, version 4


Feature importance loggée: feature_importance.png
🏃 View run RandomForestClassifier_CV_Run_Scaler_False at: http://localhost:5001/#/experiments/672741076646558312/runs/3b5ff5a4ce814c9e9623223e74d419e1
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour RandomForestClassifier terminée et loggée dans MLflow. ---


Created version '4' of model 'RandomForestClassifier_CreditScoring_CV_Scaler_False'.


In [14]:
# --- Modèle 3: XGBoost ---
xgb_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'eval_metric': 'logloss',
    'scale_pos_weight': (y_train_full == 0).sum() / (y_train_full == 1).sum(),
    "tree_method": "hist",
    "n_jobs": -1
}
xgb_avg_metrics = train_and_track_model_with_cv(
    "XGBoostClassifier", XGBClassifier, xgb_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=False  # garder le ratio, pas de SMOTE ici
)
all_models_avg_metrics["XGBoostClassifier_NoSMOTE"] = xgb_avg_metrics


--- Démarrage de l'expérimentation pour XGBoostClassifier ---
  XGBoostClassifier - Fold 1 metrics logged. ROC AUC: 0.7621
🏃 View run XGBoostClassifier_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/9c5ca9738b5342a79cb3cea894acfa14
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  XGBoostClassifier - Fold 2 metrics logged. ROC AUC: 0.7601
🏃 View run XGBoostClassifier_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/bf296ff5ab2845deb4159f175ae88b06
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  XGBoostClassifier - Fold 3 metrics logged. ROC AUC: 0.7591
🏃 View run XGBoostClassifier_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/ccc05f1a81c84572b1d727e10cc991c7
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  XGBoostClassifier - Fold 4 metrics logged. ROC AUC: 0.7632
🏃 View run XGBoostClassifier_Fold_4 at: http://localhost:5001/#

Registered model 'XGBoostClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/12/05 14:10:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostClassifier_CreditScoring_CV_Scaler_False, version 3


Feature importance loggée: feature_importance.png
🏃 View run XGBoostClassifier_CV_Run_Scaler_False at: http://localhost:5001/#/experiments/672741076646558312/runs/a5a4c28169984de78dbf62ab8169dc2c
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour XGBoostClassifier terminée et loggée dans MLflow. ---


Created version '3' of model 'XGBoostClassifier_CreditScoring_CV_Scaler_False'.


In [15]:
# --- Modèle 3: XGBoost ---
xgb_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'eval_metric': 'logloss',
    'scale_pos_weight': (y_train_full == 0).sum() / (y_train_full == 1).sum(),
    "tree_method": "hist",
    "n_jobs": -1
}
xgb_avg_metrics = train_and_track_model_with_cv(
    "XGBoostClassifier", XGBClassifier, xgb_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=True  # garder le ratio, pas de SMOTE ici
)
all_models_avg_metrics["XGBoostClassifier_SMOTE"] = xgb_avg_metrics


--- Démarrage de l'expérimentation pour XGBoostClassifier ---
  XGBoostClassifier - Fold 1 metrics logged. ROC AUC: 0.7498
🏃 View run XGBoostClassifier_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/5bcfeef962994da1b8142949962632a4
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  XGBoostClassifier - Fold 2 metrics logged. ROC AUC: 0.7474
🏃 View run XGBoostClassifier_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/53b9cd8ed73743989f9fce5d690bc116
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  XGBoostClassifier - Fold 3 metrics logged. ROC AUC: 0.7472
🏃 View run XGBoostClassifier_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/f534d5e5568d45c1b13f1b71d4cda858
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  XGBoostClassifier - Fold 4 metrics logged. ROC AUC: 0.7533
🏃 View run XGBoostClassifier_Fold_4 at: http://localhost:5001/#

Registered model 'XGBoostClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/12/05 14:11:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostClassifier_CreditScoring_CV_Scaler_False, version 4


Feature importance loggée: feature_importance.png
🏃 View run XGBoostClassifier_CV_Run_Scaler_False at: http://localhost:5001/#/experiments/672741076646558312/runs/664fcbc2e37748ab9ecc70155e8f62df
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour XGBoostClassifier terminée et loggée dans MLflow. ---


Created version '4' of model 'XGBoostClassifier_CreditScoring_CV_Scaler_False'.


In [16]:
# --- Modèle 4: LightGBM ---
lgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': True,
    'n_jobs': -1,
    'force_row_wise': True,
    'verbosity': -1
}
lgbm_avg_metrics = train_and_track_model_with_cv(
    "LightGBMClassifier", LGBMClassifier, lgbm_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=False  # garder is_unbalance
)
all_models_avg_metrics["LightGBMClassifier_NoSMOTE"] = lgbm_avg_metrics



--- Démarrage de l'expérimentation pour LightGBMClassifier ---
  LightGBMClassifier - Fold 1 metrics logged. ROC AUC: 0.7616
🏃 View run LightGBMClassifier_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/d1b4ad3406df4f35a30980f3e1779740
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LightGBMClassifier - Fold 2 metrics logged. ROC AUC: 0.7612
🏃 View run LightGBMClassifier_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/3de4239bff2c4b7fb6c16186b773f793
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LightGBMClassifier - Fold 3 metrics logged. ROC AUC: 0.7600
🏃 View run LightGBMClassifier_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/81e3b50ab1e94bf694e66187cc89ad77
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LightGBMClassifier - Fold 4 metrics logged. ROC AUC: 0.7652
🏃 View run LightGBMClassifier_Fold_4 at: http://localho

Registered model 'LightGBMClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/12/05 14:11:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LightGBMClassifier_CreditScoring_CV_Scaler_False, version 3


Feature importance loggée: feature_importance.png
🏃 View run LightGBMClassifier_CV_Run_Scaler_False at: http://localhost:5001/#/experiments/672741076646558312/runs/313768e0b7dd4deb951fa81d5062a117
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour LightGBMClassifier terminée et loggée dans MLflow. ---


Created version '3' of model 'LightGBMClassifier_CreditScoring_CV_Scaler_False'.


In [17]:
# --- Modèle 4: LightGBM ---
lgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': True,
    'n_jobs': -1,
    'force_row_wise': True,
    'verbosity': -1
}
lgbm_avg_metrics = train_and_track_model_with_cv(
    "LightGBMClassifier", LGBMClassifier, lgbm_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=True  # garder is_unbalance
)
all_models_avg_metrics["LightGBMClassifier_SMOTE"] = lgbm_avg_metrics


--- Démarrage de l'expérimentation pour LightGBMClassifier ---
  LightGBMClassifier - Fold 1 metrics logged. ROC AUC: 0.7616
🏃 View run LightGBMClassifier_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/891fe0d8f5404f4eae89918b58c43555
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LightGBMClassifier - Fold 2 metrics logged. ROC AUC: 0.7566
🏃 View run LightGBMClassifier_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/1e6831b1f4dc4d7bb43523267995412f
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LightGBMClassifier - Fold 3 metrics logged. ROC AUC: 0.7576
🏃 View run LightGBMClassifier_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/d64bcd64bad84a848eb0b9d29cc9106b
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  LightGBMClassifier - Fold 4 metrics logged. ROC AUC: 0.7620
🏃 View run LightGBMClassifier_Fold_4 at: http://localho

Registered model 'LightGBMClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/12/05 14:12:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LightGBMClassifier_CreditScoring_CV_Scaler_False, version 4


Feature importance loggée: feature_importance.png
🏃 View run LightGBMClassifier_CV_Run_Scaler_False at: http://localhost:5001/#/experiments/672741076646558312/runs/7bf9e31bc3504562b9d84352e942a1df
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour LightGBMClassifier terminée et loggée dans MLflow. ---


Created version '4' of model 'LightGBMClassifier_CreditScoring_CV_Scaler_False'.


In [18]:
# --- Modèle 5: MLP (avec StandardScaler, option SMOTE) ---
mlp_params = {
    "hidden_layer_sizes": (128, 64),
    "activation": "relu",        # ReLU: rapide, évite saturations de tanh/sigmoid
    "alpha": 1e-4,
    "learning_rate": "adaptive",
    "max_iter": 100,
    "early_stopping": True
}
mlp_avg_metrics = train_and_track_model_with_cv(
    "MLPClassifier", MLPClassifier, mlp_params,
    X_train_full, y_train_full,
    apply_scaler=True,
    use_smote=True
)
all_models_avg_metrics["MLPClassifier_Scaled_SMOTE"] = mlp_avg_metrics


--- Démarrage de l'expérimentation pour MLPClassifier ---
  MLPClassifier - Fold 1 metrics logged. ROC AUC: 0.7255
🏃 View run MLPClassifier_Fold_1 at: http://localhost:5001/#/experiments/672741076646558312/runs/f543e7a9fb2a4afc923484ee6ef95f46
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  MLPClassifier - Fold 2 metrics logged. ROC AUC: 0.7085
🏃 View run MLPClassifier_Fold_2 at: http://localhost:5001/#/experiments/672741076646558312/runs/2272b1ce062240d6b91f4002af1c5c3b
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  MLPClassifier - Fold 3 metrics logged. ROC AUC: 0.7208
🏃 View run MLPClassifier_Fold_3 at: http://localhost:5001/#/experiments/672741076646558312/runs/acb5e89609904e248f39c44ec0ccf056
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
  MLPClassifier - Fold 4 metrics logged. ROC AUC: 0.7127
🏃 View run MLPClassifier_Fold_4 at: http://localhost:5001/#/experiments/672741076646558312/runs

Registered model 'MLPClassifier_CreditScoring_CV_Scaler_True' already exists. Creating a new version of this model...
2025/12/05 14:13:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MLPClassifier_CreditScoring_CV_Scaler_True, version 2


Pas de feature_importances_ disponible.
🏃 View run MLPClassifier_CV_Run_Scaler_True at: http://localhost:5001/#/experiments/672741076646558312/runs/21ffbf53d4b1428cb24e94e8db856ead
🧪 View experiment at: http://localhost:5001/#/experiments/672741076646558312
--- Expérimentation pour MLPClassifier terminée et loggée dans MLflow. ---


Created version '2' of model 'MLPClassifier_CreditScoring_CV_Scaler_True'.


In [19]:
# --- Comparaison finale des modèles ---
print("\n--- Récapitulatif des performances moyennes (ROC AUC) ---")
for model_name, metrics in all_models_avg_metrics.items():
    print(f"  {model_name}: Avg ROC AUC = {metrics['roc_auc']:.4f}")


--- Récapitulatif des performances moyennes (ROC AUC) ---
  LogisticRegression_Scaled: Avg ROC AUC = 0.7427
  LogisticRegression_Scaled_SMOTE: Avg ROC AUC = 0.6815
  RandomForestClassifier_NoScaler: Avg ROC AUC = 0.7394
  RandomForestClassifier_SMOTE: Avg ROC AUC = 0.6756
  XGBoostClassifier_NoSMOTE: Avg ROC AUC = 0.7609
  XGBoostClassifier_SMOTE: Avg ROC AUC = 0.7493
  LightGBMClassifier_NoSMOTE: Avg ROC AUC = 0.7620
  LightGBMClassifier_SMOTE: Avg ROC AUC = 0.7592
  MLPClassifier_Scaled_SMOTE: Avg ROC AUC = 0.7149
